### User OGS games dump

This notebook generates a dump of your OGS games. It uses one of the previously created databases of games (via `stats_dump.ipynb` or `stats_clean.ipynb`) to enumerate a list of game IDs, then retrives JSON representations of each game from the OGS API, and finally converts them to SGF.

Since the endpoints we call here contain more detailed information about each game than the `https://online-go.com/api/v1/players/<player_id>/games` endpoint, we also take the opportunity to create a new dump of game information (so as to avoid calling the API for this information later).

In [ ]:
import requests
import json
import time
import random
import pandas as pd
import os
from sys import exit

Replace the following `player_id` number with the one whose games you wish to fetch. Replace the `dataset` name with the desired dataset in `./output/<player_id>/datasets`. The tool will fetch the games corresponding to the named dataset specifically.
* Note: If you create your own datasets for use with this tool, do not drop the `'related'` column during dataset creation. It is used to request the json files for the games. (Technically, all we need is the game id to identify the correct endpoint, so as long as this information is present in the dataset you should be able to modify the code to achieve the same goals.)

In [ ]:
player_id = dummy_id
dataset = 'clean_stats.json'

In [ ]:
# Obtain API authorization keys
with open('keys.txt', 'r') as f:
    lines = f.readlines()
    client_id = lines[0].split('\"')[1]
    client_secret = lines[1].split('\"')[1]
    username = lines[2].split('\"')[1]
    password = lines[3].split('\"')[1]

# Obtain authorization token
s = requests.Session()
response = s.post('https://online-go.com/oauth2/token/',data={'grant_type':'password','username':username,'password':password,'client_id':client_id,'client_secret':client_secret})
result = response.json()
token = result['access_token']
headers = {'Authorization': 'Bearer {}'.format(token)}

The following cell fetches the JSON representations of the games and stores them in the `./ogsjson` directory. The OGS API has a rate limit on requests, so there is also a very primitive rate backoff implemented. You will see some hangups in the output as the rate limit is encountered and the loop has to wait out its backoff time. This code block could take several minutes depending on how many games you've played.

In [ ]:
gamesList = [x['detail'] for x in pd.read_json('./output/'+str(player_id)+'/datasets/'+dataset)['related'].tolist()]
games_dump = []
try:
    os.mkdir('output/'+str(player_id)+'/ogsjson')
except FileExistsError:
    pass

attempts = 0
max_attempts = 10
games_fetched = 0

for x in gamesList:
    endpoint = 'https://online-go.com/api/v1/games/' + x[14:]
    response = s.get(endpoint,  headers = headers)
    while response.status_code == 429: # If rate limited, wait and then start again
        print('Rate limit reached. Backing off before retrying query. Please wait...')
        attempts += 1
        if attempts >= max_attempts:
            print('Too many attempts.')
            break
        else:
            time.sleep((2 ** attempts) + random.random())
            response = s.get(endpoint,  headers = headers)
    # If not rate limited, continue
    attempts = 0
    try:
        games_fetched += 1
        games_dump.append(response.json())
        game_json_path = './output/'+str(player_id)+'/ogsjson/'+str(response.json()['id'])+'.json'
        game_sgf_path = './output/'+str(player_id)+'/ogssgf/'+str(response.json()['id'])+'.sgf'
        games_dump[-1]['game_json_path'] = game_json_path
        games_dump[-1]['game_sgf_path'] = game_sgf_path
        with open(game_json_path, 'w', encoding='utf-8') as f:
            f.write(json.dumps(response.json()['gamedata'], indent = 2))
    except KeyError:
        print(x)
        print(response.json())
        exit('KeyError')

print(games_fetched, 'games fetched.')

This cell uses [ogstosgf](https://github.com/lightvector/ogstosgf) to convert each game to SGF format, then moves the SGFs to an output directory `./ogssgf`.

In [ ]:
jsonPath = 'output/'+str(player_id)+'/ogsjson/'
sgfPath = 'output/'+str(player_id)+'/ogssgf/'
%run -i "ogstosgf.py" $jsonPath

try:
    os.mkdir('output/'+str(player_id)+'/ogssgf')
except FileExistsError:
    pass

stats_path = './output/'+str(player_id)+'/datasets/'+dataset[:len(dataset)-5]+'_detailed.json'
with open(stats_path, 'w', encoding='utf-8') as f:
    f.write(json.dumps(games_dump, indent = 2))
    
for filename in os.listdir(jsonPath):
    if filename.endswith('.sgf'):
        os.replace(jsonPath + filename, sgfPath + filename)